In [1]:
import dask
import gev_utils as gevu
from regridding import regrid
from utils import gev_metric_ids

In [2]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [3]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    account="pches",
    # account="open",
    cores=1,
    processes=1,
    memory="100GiB",
    walltime="02:00:00",
)

cluster.scale(jobs=5)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.81:46857,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Fit GEV

In [4]:
# Stationary, L-moments, main
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = True,
        fit_method = "lmom",
        periods_for_level = [10, 25, 50, 100],
        levels_for_period = None,
        proj_years = [2050,2100],
        hist_years = [1950,2014],
        bootstrap = False
    )

CPU times: user 13.8 s, sys: 829 ms, total: 14.7 s
Wall time: 31.7 s


In [ ]:
## Stationary, L-moments, bootstrap

# Can separate STAR-ESDM since it's higher resolution 
# and requires significantly more memory
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = True,
        fit_method = "lmom",
        periods_for_level = [10, 25, 50, 100],
        levels_for_period = None,
        proj_years = [2050,2100],
        hist_years = [1950,2014],
        bootstrap = True,
        include_STAR_ESDM = True
    )

## Regridding

In [4]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 2min 33s, sys: 1min 19s, total: 3min 53s
Wall time: 8min 12s


In [11]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in gev_metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/gard_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/extreme_value/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid(ds_tmp, method=method, target=target)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 2min 11s, sys: 1min 13s, total: 3min 24s
Wall time: 9min 6s
